# Побудова регресійної моделі: підготовка та візуалізація даних

## **Лінійна регресія для гарбузів - Урок 2**
#### Вступ

Тепер, коли ви налаштували інструменти для створення моделей машинного навчання за допомогою Tidymodels і Tidyverse, ви готові почати ставити запитання до своїх даних. Працюючи з даними та застосовуючи рішення машинного навчання, дуже важливо вміти правильно формулювати запитання, щоб максимально розкрити потенціал вашого набору даних.

У цьому уроці ви дізнаєтеся:

-   Як підготувати дані для побудови моделі.

-   Як використовувати `ggplot2` для візуалізації даних.

Запитання, на яке ви хочете отримати відповідь, визначатиме, які алгоритми машинного навчання ви будете використовувати. А якість отриманої відповіді значною мірою залежатиме від характеру ваших даних.

Давайте розглянемо це на практичному прикладі.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>


<!--![Ілюстрація від \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Ілюстрація від \@allison_horst-->


## 1. Імпорт даних про гарбузи та виклик Tidyverse

Нам знадобляться наступні пакети для аналізу даних у цьому уроці:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — це [колекція пакетів для R](https://www.tidyverse.org/packages), створена для того, щоб зробити роботу з даними швидшою, простішою та цікавішою!

Ви можете встановити їх за допомогою:

`install.packages(c("tidyverse"))`

Скрипт нижче перевіряє, чи є у вас необхідні пакети для виконання цього модуля, і встановлює їх, якщо деякі відсутні.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Тепер давайте запустимо кілька пакетів і завантажимо [дані](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), надані для цього уроку!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Швидкий виклик `glimpse()` одразу показує, що є пропуски та змішання рядків (`chr`) і числових даних (`dbl`). `Date` має тип character, а також є дивний стовпець під назвою `Package`, де дані змішані між `sacks`, `bins` та іншими значеннями. Дані, насправді, трохи хаотичні 😤.

Насправді, не дуже часто отримуєш набір даних, який повністю готовий до використання для створення моделі машинного навчання "з коробки". Але не хвилюйтесь, у цьому уроці ви навчитеся готувати сирий набір даних, використовуючи стандартні бібліотеки R 🧑‍🔧. Ви також опануєте різні техніки для візуалізації даних.📈📊
<br>

> Нагадування: Оператор pipe (`%>%`) виконує операції в логічній послідовності, передаючи об'єкт далі у функцію або вираз виклику. Ви можете уявити оператор pipe як "а потім" у вашому коді.


## 2. Перевірка на відсутні дані

Однією з найпоширеніших проблем, з якими стикаються дата-сайентисти, є неповні або відсутні дані. У R відсутні або невідомі значення позначаються спеціальним значенням-сторожем: `NA` (Not Available).

Тож як ми можемо дізнатися, що у фреймі даних є відсутні значення?  
<br>
- Один із простих способів — використати базову функцію R `anyNA`, яка повертає логічні значення `TRUE` або `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Чудово, здається, деякі дані відсутні! Це гарне місце, щоб почати.

-   Інший спосіб — використати функцію `is.na()`, яка вказує, які окремі елементи стовпця відсутні, за допомогою логічного значення `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Гаразд, завдання виконано, але з таким великим фреймом даних, як цей, було б неефективно і практично неможливо переглянути всі рядки та стовпці окремо😴.

-   Більш інтуїтивним способом було б обчислити суму відсутніх значень для кожного стовпця:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Набагато краще! Даних не вистачає, але, можливо, це не матиме значення для виконання завдання. Подивимося, до чого приведе подальший аналіз.

> Окрім чудового набору пакетів і функцій, R має дуже хорошу документацію. Наприклад, використовуйте `help(colSums)` або `?colSums`, щоб дізнатися більше про функцію.


## 3. Dplyr: Граматика обробки даних

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>


<!--![Ілюстрація від \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ілюстрація від \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), пакет у Tidyverse, є граматикою маніпуляції даними, яка пропонує узгоджений набір дієслів, що допомагають вирішувати найпоширеніші завдання з обробки даних. У цьому розділі ми розглянемо деякі дієслова dplyr!


#### dplyr::select()

`select()` — це функція з пакету `dplyr`, яка допомагає вибирати стовпці для збереження або виключення.

Щоб зробити ваш датафрейм зручнішим для роботи, видаліть кілька його стовпців, використовуючи `select()`, залишивши лише ті, які вам потрібні.

Наприклад, у цьому завданні наш аналіз включатиме стовпці `Package`, `Low Price`, `High Price` та `Date`. Давайте виберемо ці стовпці.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` — це функція з пакету `dplyr`, яка допомагає створювати або змінювати стовпці, зберігаючи при цьому існуючі стовпці.

Загальна структура `mutate` виглядає так:

`data %>%   mutate(new_column_name = what_it_contains)`

Давайте випробуємо `mutate`, використовуючи стовпець `Date`, виконавши наступні операції:

1.  Перетворимо дати (які наразі мають тип character) у формат місяця (це дати у форматі США, тому формат — `MM/DD/YYYY`).

2.  Витягнемо місяць із дат у новий стовпець.

У R пакет [lubridate](https://lubridate.tidyverse.org/) спрощує роботу з даними типу Date-time. Тож давайте використаємо `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` і подивимося, як досягти зазначених цілей. Ми можемо видалити стовпець Date, оскільки він більше не знадобиться в подальших операціях.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ура! 🤩

Тепер давайте створимо нову колонку `Price`, яка представляє середню ціну гарбуза. Для цього візьмемо середнє значення колонок `Low Price` та `High Price`, щоб заповнити нову колонку Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Так!💪

"Стривайте!", скажете ви, переглянувши весь набір даних за допомогою `View(pumpkins)`, "Тут щось дивне!"🤔

Якщо подивитися на стовпець `Package`, гарбузи продаються в багатьох різних конфігураціях. Деякі продаються в мірах `1 1/9 bushel`, деякі в мірах `1/2 bushel`, деякі поштучно, деякі за вагу, а деякі в великих коробках з різною шириною.

Давайте перевіримо це:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Дивовижно!👏

Гарбузи, здається, дуже важко зважувати стабільно, тому давайте відфільтруємо їх, вибравши лише ті гарбузи, у яких у стовпці `Package` є рядок *bushel*, і помістимо це в новий датафрейм `new_pumpkins`.


#### dplyr::filter() та stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): створює підмножину даних, яка містить лише **рядки**, що відповідають вашим умовам, у цьому випадку гарбузи зі словом *bushel* у стовпці `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): визначає наявність або відсутність шаблону в рядку.

Пакет [`stringr`](https://github.com/tidyverse/stringr) надає прості функції для поширених операцій з рядками.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Ви можете побачити, що ми звузили вибір до приблизно 415 рядків даних, які містять гарбузи за бушель.🤩  
<br>


#### dplyr::case_when()

**Але зачекайте! Є ще одна річ, яку потрібно зробити**

Ви помітили, що кількість бушелів змінюється в кожному рядку? Вам потрібно нормалізувати ціни, щоб показувати вартість за бушель, а не за 1 1/9 або 1/2 бушеля. Час зробити трохи математики, щоб стандартизувати це.

Ми використаємо функцію [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), щоб *змінити* колонку Price залежно від певних умов. `case_when` дозволяє векторизувати кілька операторів `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Тепер ми можемо проаналізувати ціни за одиницю, базуючись на їх вимірюванні в бушелях. Уся ця робота з бушелями гарбузів, однак, показує, наскільки `важливо` `розуміти природу ваших даних`!

> ✅ Згідно з [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), вага бушеля залежить від типу продукту, оскільки це об'ємне вимірювання. "Бушель помідорів, наприклад, має важити 56 фунтів... Листя та зелень займають більше місця з меншою вагою, тому бушель шпинату важить лише 20 фунтів." Це все досить складно! Давайте не будемо займатися перетворенням бушелів у фунти, а замість цього визначимо ціну за бушель. Уся ця робота з бушелями гарбузів, однак, показує, наскільки важливо розуміти природу ваших даних!
>
> ✅ Ви помітили, що гарбузи, які продаються за півбушеля, дуже дорогі? Можете здогадатися, чому? Підказка: маленькі гарбузи набагато дорожчі за великі, ймовірно, тому що їх набагато більше в одному бушелі, враховуючи невикористаний простір, який займає один великий порожнистий гарбуз для пирога.


Тепер, нарешті, заради пригоди 💁‍♀️, давайте також перемістимо стовпець Month на першу позицію, тобто `перед` стовпцем `Package`.

Функція `dplyr::relocate()` використовується для зміни позицій стовпців.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Молодець!👌 Тепер у вас є чистий, впорядкований набір даних, на основі якого ви можете створити нову регресійну модель!  
<br>


## 4. Візуалізація даних за допомогою ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Інфографіка від Dasani Madipalli</figcaption>


<!--![Інфографіка від Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Існує *мудрий* вислів, який звучить так:

> "Простий графік приніс більше інформації в розум аналітика даних, ніж будь-який інший інструмент." --- Джон Тьюкі

Частина ролі дата-сайєнтиста полягає в тому, щоб продемонструвати якість і характер даних, з якими він працює. Для цього вони часто створюють цікаві візуалізації, або графіки, діаграми та схеми, які показують різні аспекти даних. Таким чином, вони можуть візуально показати взаємозв’язки та прогалини, які інакше важко виявити.

Візуалізації також можуть допомогти визначити, який метод машинного навчання найбільш підходить для даних. Наприклад, точковий графік, який здається таким, що слідує за лінією, вказує на те, що дані добре підходять для завдання лінійної регресії.

R пропонує кілька систем для створення графіків, але [`ggplot2`](https://ggplot2.tidyverse.org/index.html) є однією з найелегантніших і найуніверсальніших. `ggplot2` дозволяє створювати графіки, **комбінуючи незалежні компоненти**.

Давайте почнемо з простого точкового графіка для стовпців Price і Month.

Отже, у цьому випадку ми почнемо з [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), надамо набір даних і естетичне відображення (за допомогою [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), а потім додамо шари (наприклад, [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) для точкових графіків.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Чи є це корисним графіком 🤷? Чи щось у ньому вас дивує?

Він не є особливо корисним, оскільки лише показує ваші дані як розподіл точок за певний місяць.
<br>


### **Як зробити це корисним?**

Щоб графіки відображали корисні дані, зазвичай потрібно якось згрупувати ці дані. Наприклад, у нашому випадку, визначення середньої ціни гарбузів за кожен місяць надасть більше розуміння прихованих закономірностей у наших даних. Це приводить нас до ще одного швидкого огляду **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Групову агрегацію в R можна легко виконати за допомогою

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` змінює одиницю аналізу з усього набору даних на окремі групи, наприклад, за місяцями.

-   `dplyr::summarize()` створює новий набір даних із одним стовпцем для кожної змінної групування та одним стовпцем для кожної з обчислених статистик.

Наприклад, ми можемо використати `dplyr::group_by() %>% summarize()` для групування гарбузів за стовпцем **Month**, а потім знайти **середню ціну** для кожного місяця.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Лаконічно!✨

Категоріальні ознаки, такі як місяці, краще представляти за допомогою стовпчастої діаграми 📊. Шари, які відповідають за створення стовпчастих діаграм, це `geom_bar()` і `geom_col()`. Зверніться до `?geom_bar`, щоб дізнатися більше.

Давайте створимо одну!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Це більш корисна візуалізація даних! Здається, вона показує, що найвища ціна на гарбузи спостерігається у вересні та жовтні. Чи відповідає це вашим очікуванням? Чому так або чому ні?

Вітаю з завершенням другого уроку 👏! Ви підготували свої дані для побудови моделі, а потім виявили більше інсайтів за допомогою візуалізацій!



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ мовою оригіналу слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
